### Logistic Regression

In [2]:
%run ml_helpers.ipynb
%run ml.ipynb

               ID  Domestic  Beat  Year Month  Week  Day  Hour   Watch  PRCP  \
994995   11448695     False  2422  2018     9    37   16    22   Third  0.00   
1258775  11830274     False  1653  2019     9    38   16    12  Second  0.00   
1122134  11629481     False  1113  2019     3    12   20    18   Third  0.14   
1017626  11479699     False  1134  2018    10    42   17    14  Second  0.00   
1180423  11715677     False   623  2019     6    23    8    17   Third  0.00   
1331476  11937452      True  2521  2019    12     1   31     9  Second  0.06   
1111615  11613129     False  2523  2019     3    10    4    16   Third  0.00   
1293054  11881191      True   632  2019    11    44    3    13  Second  0.00   
874453   11290691     False   215  2018     4    16   19    10  Second  0.03   
991795   11444524     False  1913  2018     9    37   12    21   Third  0.00   
883349   11301044     False   624  2018     5    18    1     9  Second  0.00   
1119424  11625054     False   112  2019 

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
<ipython-input-2-bc70820a9afc>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Year"]=df["Year"].astype("category")
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
               ID  Domestic  Beat  Year Month  Week  Day  Hour   Watch  PRCP  \
1160180  11686153     False  1431  2019     5    20   13    18   Third  0.00   
1082696  11570578     False  1434  2019     1     3   17     1   First  0.00   
1096334  11590180      True   633  2019     2     6    9     0   First  0.00   
1266485  11841868     False  1931  2019     9    39   26    16   Third  0.00   
962697   11406568     False  2212  2018     8    32    7    10  Second  0.30   
850541   11257157     False  1931  2018     3    11   15    15  Second  0.00   
1114031  11617307     False  1132  2019     3    10    8    16   Third  0.00   
1252713  11821347     False  1232  2019     9    36    8    15  Second  0.05   
1114951  11618394      True  2223  2019     3    10   10     1   First  0.00   
1172981  11705131     False  1031  2019     5  

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
<ipython-input-2-bc70820a9afc>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Year"]=df["Year"].astype("category")
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us


Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
         Beat  Year  Month   Watch  PRCP  SNOW  TMAX  TMIN  count_l_stops  \
1217530   332  2019      7  Second  0.00   0.0    86    63            0.0   
1083455   524  2019      1  Second  0.02   0.3    32    29            0.0   
1318532   132  2019     12   Third  0.00   0.2    25    14            0.0   
1320151   122  2019     12   Third  0.00   0.0    45    34            3.0   
1272106   113  2019     10  Second  0.00   0.0    59    48            4.0   
...       ...   ...    ...     ...   ...   ...   ...   ...            ...   
1099118   611  2019      2  Second  0.00   0.1    30    13            0.0   
1163093  1812  2019      5   Third  0.14   0.0    69    46            2.0   
1329222   422  2019     12   Third  0.00   0.0    40    30            0.0   
1279072  1931  2019     10   First  0.00   0.0    55    39            0.0   
1291129  1024  2019     10   Third

Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2015]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...


In [ ]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
import numpy as np
import sklearn as sk
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score
import datetime